In [31]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
import re
from pprint import pprint

### 1 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта hh.ru

In [225]:
def query(link,params={}):
    """
    Функция содержит общие параметры необходимые для запроса. 
    link – хранит ссылку на сайт, на который адресован запрос.
    params  - необязательный параметр, содержит передаваемые параметры, параметры можно передать и в ссылке link, а params оставить не заполненным.
    Возвращает функция DOM дерево.
    """
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
          'Accept':'*/*'}
    if params:
        response = requests.get(link,headers=header,params=params)
        print(response.url)
        response = response.text
    else:
        response = requests.get(link,headers=header).text
   
    soup = bs(response,'lxml')    
    return soup



def getSalary(s):
    
    """
    Функция обрабатывает строки содержащие величину заработной платы и возвращает dist, содержащий цены в структурированном виде.
    """
    
    obj = {"min":None,
           "max":None,
           "currency_unit":None
          }
    if not s:
        return obj
    q = re.sub('\s*', "", s)
    result = re.findall(r'([^0-9]*)(?:(\d*)\W)*(\d+)(\w+)', q)
    if result:
        result = result[0]
        obj["currency_unit"] = result[3].upper()
        if result[0].upper() == "от".upper():        
            obj["min"] = float(result[2])
        elif result[0].upper() == "до".upper():      
            obj["max"] = float(result[2])
        elif result[0] == "":
            obj["min"] = float(result[1])
            obj["max"] = float(result[2])           
    return obj
    


In [226]:
main_link = 'https://murmansk.hh.ru/search/vacancy?text=data+scientist'
base_link = 'https://murmansk.hh.ru'
params = {'L_is_autosearch':'false',
            'clusters':'true',
            'enable_snippets':'true',           
            'page':0}

In [227]:
hh = []
while True:
    soup = query(link=main_link,params=params)
    vacancy_serpclass = soup.find('div', {'class':'vacancy-serp'})
    vacancies = vacancy_serpclass.select(".vacancy-serp-item")
    pprint(len(vacancies))    
    button = soup.find('a', {"data-qa":"pager-next"})#кнопка ДАЛЕЕ    
    
    for item in vacancies:
        a = item.find('a', {"data-qa":"vacancy-serp__vacancy-title"})
        site = item.find('a', {"data-qa":"vacancy-serp__vacancy-employer"})        
        salary = item.find('span', {"data-qa":"vacancy-serp__vacancy-compensation"})
        salary = salary.text if salary else None
        salary = getSalary(salary)
        obj = {'specialty':a.string,
               'salary_max':salary['max'],
               'salary_min':salary['min'],
               'salary_unit':salary['currency_unit'],
               'link':a.get("href"),
               'site':base_link+site.get("href") if site else None,
               'site_job':'HH',
               '_id':str(re.findall(r'/(\d+)\?', a.get("href"))[0])+'hh'}        
        for item 
        hh.append(obj)
        
    if not button:  #проверяю наличие, кнопку ДАЛЕЕ
        break
    
    params["page"] = params["page"] + 1

https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=0
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=1
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=2
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=3
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=4
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=5
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=6
37


### 2 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru

In [228]:
main_link = 'https://russia.superjob.ru/vakansii/programmist-python.html'
base_link = 'https://russia.superjob.ru'
params = {'page':1}
soup = query(link=main_link,params=params)

https://russia.superjob.ru/vakansii/programmist-python.html?page=1


In [229]:
superjob = []
while True:
    soup = query(link=main_link,params=params)
    vacancies = soup.select(".iJCa5.f-test-vacancy-item._1fma_._1JhPh._2gFpt._1znz6._2nteL")
    pprint(len(vacancies)) 
    button = soup.select('.icMQ_._1_Cht._3ze9n.f-test-button-dalshe.f-test-link-Dalshe')#кнопка ДАЛЕЕ   
    for item in vacancies:
        a = item.select(".icMQ_._6AfZ9")[0]        
        site = item.select('.icMQ_._205Zx')
        site = site[0] if site else None        
        salary = item.select('._3mfro._2Wp8I.PlM3e._2JVkc._2VHxz')
        salary = salary[0].text if salary else None
        salary = getSalary(salary)        
        obj = {'specialty':a.text,
               'salary_max':salary['max'],
               'salary_min':salary['min'],
               'salary_unit':salary['currency_unit'],
               'link':base_link+a.get("href"),
               'site':base_link+site.get("href") if site else None,
               'site_job':'superjob',
               '_id':str(re.findall(r'-(\d+)\.', a.get("href"))[0])+'superjob'}
        superjob.append(obj)
    if not button:#проверяю наличие, кнопку ДАЛЕЕ
        break
    params["page"] = params["page"] + 1

https://russia.superjob.ru/vakansii/programmist-python.html?page=1
20
https://russia.superjob.ru/vakansii/programmist-python.html?page=2
4


In [64]:
client = MongoClient('localhost', 27017)
db = client['vacancies']

In [314]:
db.hh.delete_many({})
db.superjob.delete_many({})

In [315]:
add_new_vacancies(hh)

Добавил в базу hh
[{'_id': '37732722hh',
  'link': 'https://murmansk.hh.ru/vacancy/37732722?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/778329?dpt=778329-778329-01',
  'site_job': 'HH',
  'specialty': 'Специалист по углубленной аналитике данных (Data scientist)'},
 {'_id': '37844183hh',
  'link': 'https://murmansk.hh.ru/vacancy/37844183?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/1128487',
  'site_job': 'HH',
  'specialty': 'Principal Software Architect (Bioinformatics)'},
 {'_id': '37762953hh',
  'link': 'https://murmansk.hh.ru/vacancy/37762953?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/841087',
  'site_job': 'HH',
  'specialty': 'Senior Data Engineer / Python'},
 {'_id': '37635578hh',
  'link': 'https://murmansk.hh

  'link': 'https://murmansk.hh.ru/vacancy/37638313?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/3093773',
  'site_job': 'HH',
  'specialty': 'Аналитик данных (Data analyst/Data Scientist)'},
 {'_id': '37171113hh',
  'link': 'https://murmansk.hh.ru/vacancy/37171113?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/1587',
  'site_job': 'HH',
  'specialty': 'Data Scientist (Middle/ Senior)'},
 {'_id': '37564168hh',
  'link': 'https://murmansk.hh.ru/vacancy/37564168?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/4518364',
  'site_job': 'HH',
  'specialty': 'Data Scientist'},
 {'_id': '37666597hh',
  'link': 'https://murmansk.hh.ru/vacancy/37666597?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit':

  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/3842701',
  'site_job': 'HH',
  'specialty': 'Senior Data Engineer'},
 {'_id': '37224708hh',
  'link': 'https://murmansk.hh.ru/vacancy/37224708?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/78638',
  'site_job': 'HH',
  'specialty': 'DevOps (Hadoop)'},
 {'_id': '37506599hh',
  'link': 'https://murmansk.hh.ru/vacancy/37506599?query=data%20scientist',
  'salary_max': 1000000.0,
  'salary_min': 500000.0,
  'salary_unit': 'РУБ',
  'site': None,
  'site_job': 'HH',
  'specialty': 'Директор по инвестициям (Family office)'},
 {'_id': '37182202hh',
  'link': 'https://murmansk.hh.ru/vacancy/37182202?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/63225',
  'site_job': 'HH',
  'specialty': 'Data scientist / Machine

  'specialty': 'Data Scientist (Yaroslavl plant)'},
 {'_id': '37695969hh',
  'link': 'https://murmansk.hh.ru/vacancy/37695969?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/1966740',
  'site_job': 'HH',
  'specialty': 'Medical Advisor'},
 {'_id': '37693115hh',
  'link': 'https://murmansk.hh.ru/vacancy/37693115?query=data%20scientist',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'site': 'https://murmansk.hh.ru/employer/3529?dpt=3529-3529-prof',
  'site_job': 'HH',
  'specialty': 'Руководитель направления (бизнес-аналитик, CJE)'},
 {'_id': '37675980hh',
  'link': 'https://murmansk.hh.ru/vacancy/37675980?query=data%20scientist',
  'salary_max': 3500.0,
  'salary_min': 2500.0,
  'salary_unit': 'USD',
  'site': 'https://murmansk.hh.ru/employer/3289413',
  'site_job': 'HH',
  'specialty': 'Senior Software Engineer (Python/Devops) for a Swiss Startup'},
 {'_id': '37662634hh',
  'li

BulkWriteError: batch op errors occurred

In [316]:
len(list(db.hh.find({})))

102

In [307]:
hh[0:105]

[{'specialty': 'Специалист по углубленной аналитике данных (Data scientist)',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'link': 'https://murmansk.hh.ru/vacancy/37732722?query=data%20scientist',
  'site': 'https://murmansk.hh.ru/employer/778329?dpt=778329-778329-01',
  'site_job': 'HH',
  '_id': '37732722hh'},
 {'specialty': 'Principal Software Architect (Bioinformatics)',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'link': 'https://murmansk.hh.ru/vacancy/37844183?query=data%20scientist',
  'site': 'https://murmansk.hh.ru/employer/1128487',
  'site_job': 'HH',
  '_id': '37844183hh'},
 {'specialty': 'Senior Data Engineer / Python',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'link': 'https://murmansk.hh.ru/vacancy/37762953?query=data%20scientist',
  'site': 'https://murmansk.hh.ru/employer/841087',
  'site_job': 'HH',
  '_id': '37762953hh'},
 {'specialty': 'Data Scientist',
  'salary_max': 180000.0,
  'salary_mi

In [305]:
#db.hh.insert_many(hh)
for item in hh:
    db.hh.insert_one(item)

DuplicateKeyError: E11000 duplicate key error collection: vacancies.hh index: _id_ dup key: { _id: "37003152hh" }

In [ ]:
list(db.hh.find({}))

In [239]:
db.superjob.insert_many(superjob)

### 2. Написать  функцию,  которая  производит  поиск  и  выводит  на  экран  вакансии  с  заработной платой больше введённой суммы

In [ ]:
def get_wage(num): 
    '''
    функция, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы num
    '''
    guery = {"$or":[{"salary_min":{'$gt':num}},{'salary_min':{"$gt":num}}]}
    hh = list(db.hh.find(guery))
    superjob =list(db.superjob.find(guery))
    return hh + superjob
    for item in hh:
        pprint(item)
        
get_wage(100000)

### 3. Написать  функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [313]:
def add_new_vacancies(obj_vacancies):
    res_vacancies = set([s['_id'] for s in obj_vacancies])
    hh = list(db.hh.find({}))
    superjob =list(db.superjob.find({}))
    res_base = [s['_id'] for s in hh + superjob]
    res_vacancies_set = set(res_vacancies)
    res_base_set = set(res_base)
    new_vacancies = res_vacancies_set - res_base_set
    
    new_hh = [t for t in obj_vacancies]
    res_hh = []
    res_superjob = []
    for item in obj_vacancies:        
        if item["_id"] in new_vacancies:            
            res_hh.append(item)
    if res_hh:
        print("Добавил в базу hh")
        pprint(res_hh)
        db.hh.insert_many(res_hh)
            
    for item in obj_vacancies:       
        if item["_id"] in new_vacancies:            
            res_superjob.append(item)
    if res_superjob:
        print("Добавил в базу superjob")
        pprint(res_hh)
        db.superjob.insert_many(res_hh)
        
    return new_vacancies

In [282]:
add_new_vacancies(hh)

set()

In [281]:
hh
superjob

[{'specialty': 'Разработчик (программист) Python',
  'salary_max': 80000.0,
  'salary_min': 65000.0,
  'salary_unit': 'РУБ',
  'link': 'https://russia.superjob.ru/vakansii/razrabotchik-34018176.html',
  'site': 'https://russia.superjob.ru/clients/resurs-3889130.html',
  'site_job': 'superjob',
  '_id': '34018176superjob'},
 {'specialty': 'Программист Python',
  'salary_max': None,
  'salary_min': 70000.0,
  'salary_unit': 'РУБ',
  'link': 'https://russia.superjob.ru/vakansii/programmist-python-34009801.html',
  'site': 'https://russia.superjob.ru/clients/tenzor-430715.html',
  'site_job': 'superjob',
  '_id': '34009801superjob'},
 {'specialty': 'Full-stack-веб-программист python/javascript',
  'salary_max': None,
  'salary_min': None,
  'salary_unit': None,
  'link': 'https://russia.superjob.ru/vakansii/full-stack-veb-programmist-python-33998693.html',
  'site': 'https://russia.superjob.ru/clients/fgbu-centr-strategicheskogo-planirovaniya-mediko-biologicheskih-riskov-fmba-rf-3776154.ht

In [279]:
rr = db.hh.find({"_id":'35653703hh'})
for item in rr:
    pprint(item)

{'_id': '35653703hh',
 'link': 'https://murmansk.hh.ru/vacancy/35653703?query=data%20scientist',
 'salary_max': None,
 'salary_min': None,
 'salary_unit': None,
 'site': 'https://murmansk.hh.ru/employer/3529?dpt=3529-3529-prof',
 'site_job': 'HH',
 'specialty': 'Инженер/Data-Scientist'}


'33846983'

In [ ]:
for item in db.hh.find({"$or":[{"salary_min":{'$gt':100000}},{'salary_min':{"$gt":100000}}]}):
    pprint(item)

In [197]:
w = db.hh.find({},{'_id':1})
#for item in w:
#    pprint(item['_id'])
    
    
    
a1=[i['_id'] for i in w]   

pprint(a1)

['37732722hh',
 '37762953hh',
 '37635578hh',
 '37809212hh',
 '37821046hh',
 '37821047hh',
 '37521387hh',
 '37698748hh',
 '37572417hh',
 '37559909hh',
 '37681207hh',
 '37569615hh',
 '37369353hh',
 '37221045hh',
 '37143803hh',
 '37782747hh',
 '37706122hh',
 '37821820hh',
 '37624725hh',
 '37807324hh',
 '37662353hh',
 '37813468hh',
 '36997030hh',
 '37457936hh',
 '37473937hh',
 '36023475hh',
 '37761187hh',
 '37798237hh',
 '36997016hh',
 '37716881hh',
 '37327686hh',
 '37732205hh',
 '37800227hh',
 '37840311hh',
 '37405358hh',
 '36801563hh',
 '37796905hh',
 '37092576hh',
 '37565065hh',
 '37758127hh',
 '34982073hh',
 '37830691hh',
 '37830692hh',
 '36861139hh',
 '37807237hh',
 '37778492hh',
 '37715784hh',
 '37332190hh',
 '35894037hh',
 '37751628hh',
 '37454715hh',
 '37282460hh',
 '37522593hh',
 '37416597hh',
 '36519811hh',
 '37547541hh',
 '37686393hh',
 '37481641hh',
 '36783102hh',
 '36667663hh',
 '37639273hh',
 '36719374hh',
 '37454742hh',
 '37492807hh',
 '37402939hh',
 '37732669hh',
 '36877389

In [198]:
from urllib.parse import urlparse
r1 = db.hh.find({},{'link':1, '_id':0})

c=[re.findall(r'/(\d+)\?', t['link'])[0]+'hh' for t in r1]
c[10] in a1

True

In [146]:
'37733784' in c

True

In [141]:
result

['37565065']

In [199]:
list1 = ['37565075hh1',
 '37758177h1',
 '34982073h1',
 '37830671h1',
 '37830672h1',
 '36861179h1',
 '37807277h1',
 '37778472h1',
 '37715774h1',
 '37332170h1h',
 '35894077h1h',
 '37751678h1h',
 '37454775h1h',
 '37282460hh',
 '37522593hh',
 '37416597hh',
 '36519811hh',
 '37547541hh',
 '37686393hh',
 '37481641hh',
 '36783102hh',
 '36667663hh',
 '37639273hh',
 '36719374hh',
 '37454742hh',
 '37492807hh',
 '37402939hh',
 '37732669hh',
 '36877389hh',
 '36999290hh',
 '37692067hh',
 '37658350hh']

In [201]:
k_new = set(list1)
k_ = set(c)

In [204]:
res_k_new = k_new - k_ # множество из всех элементов k_new, не принадлежащие ни одному из k_.
res_k_new

{'34982073h1',
 '35894077h1h',
 '36861179h1',
 '37332170h1h',
 '37454775h1h',
 '37565075hh1',
 '37715774h1',
 '37751678h1h',
 '37758177h1',
 '37778472h1',
 '37807277h1',
 '37830671h1',
 '37830672h1'}

In [205]:
k_new - res_k_new

{'36519811hh',
 '36667663hh',
 '36719374hh',
 '36783102hh',
 '36877389hh',
 '36999290hh',
 '37282460hh',
 '37402939hh',
 '37416597hh',
 '37454742hh',
 '37481641hh',
 '37492807hh',
 '37522593hh',
 '37547541hh',
 '37639273hh',
 '37658350hh',
 '37686393hh',
 '37692067hh',
 '37732669hh'}

In [ ]:
#for item in db.vacancies.find({"$or":[{"hh.salary_min":{'$lt':100000}},{'hh.salary_max':{"$lt":100000}}]},{"hh.salary_min":1,'hh.salary_max':1}):
#    pprint(item)
    
for item in db.vacancies.find({"$or":[{"hh.salary_min":100000},{'hh.salary_max':100000}]},{"hh.salary_min":1,'hh.salary_max':1}):
    pprint(item)